In [1]:
%load_ext autotime
import sys
sys.path.append('/Users/pradap/Documents/Research/Python-Package/anhaid/py_entitymatching/')

In [2]:
import py_entitymatching as em

time: 1.33 s


In [3]:
A = em.load_dataset('person_table_A', key='ID')
B = em.load_dataset('person_table_B', key='ID')

time: 5.85 ms


In [4]:
A

,ID,name,birth_year,hourly_wage,address,zipcode
0,a1,Kevin Smith,1989,30.0,"607 From St, San Francisco",94107
1,a2,Michael Franklin,1988,27.5,"1652 Stockton St, San Francisco",94122
2,a3,William Bridge,1986,32.0,"3131 Webster St, San Francisco",94107
3,a4,Binto George,1987,32.5,"423 Powell St, San Francisco",94122
4,a5,Alphonse Kemper,1984,35.0,"1702 Post Street, San Francisco",94122


time: 40.4 ms


In [5]:
B

,ID,name,birth_year,hourly_wage,address,zipcode
0,b1,Mark Levene,1987,29.5,"108 Clement St, San Francisco",94107
1,b2,Bill Bridge,1986,32.0,"3131 Webster St, San Francisco",94107
2,b3,Mike Franklin,1988,27.5,"1652 Stockton St, San Francisco",94122
3,b4,Joseph Kuan,1982,26.0,"108 South Park, San Francisco",94122
4,b5,Alfons Kemper,1984,35.0,"170 Post St, Apt 4, San Francisco",94122
5,b6,Michael Brodie,1987,32.5,"133 Clement Street, San Francisco",94107


time: 21.4 ms


In [6]:
L = em.read_csv_metadata('../../../garage/person_labeled_data.csv', key='_id', 
                         fk_ltable='ltable_ID', fk_rtable='rtable_ID',
                        ltable=A, rtable=B)

time: 4.12 ms


In [7]:
from py_entitymatching.utils.generic_helper import list_diff

time: 668 µs


In [8]:
em.get_key(A)

'ID'

time: 2.58 ms


In [9]:
A1 = A[list_diff(list(A.columns), [em.get_key(A)])]
B1 = B[list_diff(list(B.columns), [em.get_key(B)])]

time: 3.27 ms


In [10]:
feature_table = em.get_features_for_matching(A1, B1)

time: 41.1 ms


In [11]:
em.set_fk_ltable(L, 'ltable_ID')
em.set_fk_rtable(L, 'rtable_ID')
em.set_ltable(L, A)
em.set_rtable(L, B)

True

time: 9.13 ms


In [12]:
E = em.extract_feature_vecs(L, feature_table=feature_table, attrs_after='label')

0%  100%
[######] | ETA: 00:00:00

time: 62 ms



Total time elapsed: 00:00:00


In [13]:
E.head()

,_id,ltable_ID,rtable_ID,name_name_jac_qgm_3_qgm_3,name_name_cos_dlm_dc0_dlm_dc0,name_name_jac_dlm_dc0_dlm_dc0,name_name_mel,name_name_lev_dist,name_name_lev_sim,name_name_nmw,...,birth_year_birth_year_lev_sim,hourly_wage_hourly_wage_exm,hourly_wage_hourly_wage_anm,hourly_wage_hourly_wage_lev_dist,hourly_wage_hourly_wage_lev_sim,zipcode_zipcode_exm,zipcode_zipcode_anm,zipcode_zipcode_lev_dist,zipcode_zipcode_lev_sim,label
0,0,a2,b3,0.500000,0.5,0.333333,0.823932,4,0.750000,9.0,...,1.00,1,1.000000,0.0,1.00,1,1.000000,0.0,1.0,1
1,1,a3,b2,0.380952,0.5,0.333333,0.807792,4,0.714286,7.0,...,1.00,1,1.000000,0.0,1.00,1,1.000000,0.0,1.0,0
2,2,a5,b5,0.454545,0.5,0.333333,0.859487,3,0.800000,10.0,...,1.00,1,1.000000,0.0,1.00,1,1.000000,0.0,1.0,1
3,3,a1,b1,0.000000,0.0,0.000000,0.393939,11,0.000000,0.0,...,0.75,0,0.983333,3.0,0.25,1,1.000000,0.0,1.0,0
4,4,a2,b2,0.000000,0.0,0.000000,0.501263,12,0.250000,-1.0,...,0.75,0,0.859375,3.0,0.25,0,0.999841,2.0,0.6,0


time: 53.8 ms


In [14]:
from py_entitymatching.matcherselector.mlmatcherselection import get_xy_data

time: 1.44 ms


In [15]:
x,y=get_xy_data(x=None, y=None, table=E, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], target_attr='label')

time: 7.69 ms


In [16]:
from sklearn.cross_validation import KFold, cross_val_score

time: 1.41 ms


In [17]:
k=5
random_state=None
cv = KFold(len(y), k, shuffle=True, random_state=random_state)

time: 3.95 ms


In [18]:
for t, s in cv:
    print("%s, %s" %(t, s))

[0 1 3 5], [2 4]
[0 1 2 4 5], [3]
[1 2 3 4 5], [0]
[0 2 3 4 5], [1]
[0 1 2 3 4], [5]
time: 9.56 ms


In [19]:
from sklearn import cross_validation

time: 797 µs


In [20]:
rf = em.RFMatcher()

time: 903 µs


In [21]:
# predicted = cross_validation.cross_val_predict(rf.clf, x, y, cv=5)

time: 812 µs


In [22]:
from sklearn.ensemble import RandomForestClassifier

time: 1.1 ms


In [23]:
clf = RandomForestClassifier(n_estimators=11, max_depth=None,
    min_samples_split=1, random_state=0)

time: 1.31 ms


In [24]:
for train_index, test_index in cv:
    tr_x = x[train_index]
    tr_y = y[train_index]
    ts_x = x[test_index]
    ts_y = y[test_index]

time: 3.67 ms


In [25]:
ts_y, tr_y

(array([0]), array([1, 0, 1, 0, 0]))

time: 4.55 ms


In [26]:
classifier = clf.fit(tr_x, tr_y)
predictions = classifier.predict_proba(ts_x)
pr_y = classifier.predict(ts_x)
print(predictions)
print(pr_y)

[[ 1.  0.]]
[0]
time: 55.5 ms


In [27]:
import pandas as pd

time: 711 µs


In [28]:
a1 = L.loc[test_index]

time: 2.6 ms


In [29]:
b1 = pd.DataFrame({'predictions':pr_y, 'prob_false':predictions[:,0], 'prob_true':predictions[:,1]})

time: 4.1 ms


In [30]:
b1.index = a1.index

time: 701 µs


In [31]:
pd.concat([a1, b1], axis=1)

,_id,ltable_ID,rtable_ID,label,predictions,prob_false,prob_true
5,5,a2,b4,0,0,1.0,0.0


time: 18.3 ms


In [32]:
train_index, test_index

(array([0, 1, 2, 3, 4]), array([5]))

time: 3.74 ms


In [33]:
feats = list_diff(list(E.columns), ['_id', 'ltable_ID', 'rtable_ID', 'label'])

time: 1.2 ms


In [34]:
E1 = E.loc[train_index][feats]
E2 = E.loc[train_index]['label']
G1 = E.loc[test_index][feats]
G2 = E.loc[test_index]['label']

time: 7.25 ms


In [35]:
classifier = clf.fit(E1, E2)

time: 12.7 ms


In [36]:
predictions = classifier.predict_proba(G1)
# pr_y = classifier.predict(ts_x)
# print(predictions)
# print(pr_y)

time: 13.8 ms


In [37]:
pd.DataFrame(predictions)

,0,1
0,1.0,0.0


time: 27.1 ms


In [38]:
pr_y = classifier.predict(G1)

time: 3.22 ms
